# **The Small Long: Real Estate Multi Regression Project**

This project aims to explore how to predict the prices of various real estate properties in the given dataset location by assessing the following variables: lot area, square footage, bed/bath numbers, garage capacity, wood deck square footage, year built, and year remodeled. This is a multiple regression problem with SalePrice as the target variable and property characteristics as predictors. Understanding these relationships helps us prepare for the future, when we’ll need to evaluate real estate and decide which properties are worth investing in.


### **Data Preparation**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
HousingPrices = pd.read_csv("Data/Housing_Prices_24.csv")
HousingPrices.head(5)

,Id,SalePrice,LotArea,TotalBsmtSF,1stFlrSF,2ndFlrSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,...,PoolArea,LotConfig,BldgType,YearBuilt,YearRemodAdd,Foundation,CentralAir,KitchenQual,PavedDrive,YrSold
0,1,208500,8450,856,856,854,2,1,3,8,...,0,Inside,1Fam,2003,2003,PConc,Y,Gd,Y,2008
1,2,181500,9600,1262,1262,0,2,0,3,6,...,0,FR2,1Fam,1976,1976,CBlock,Y,TA,Y,2007
2,3,223500,11250,920,920,866,2,1,3,6,...,0,Inside,1Fam,2001,2002,PConc,Y,Gd,Y,2008
3,4,140000,9550,756,961,756,1,0,3,7,...,0,Corner,1Fam,1915,1970,BrkTil,Y,Gd,Y,2006
4,5,250000,14260,1145,1145,1053,2,1,4,9,...,0,FR2,1Fam,2000,2000,PConc,Y,Gd,Y,2008


### **Visualization*3**

### **Regression Model**

### **Hypothesis Test**